In [1]:
import os
import sys
import mlflow
import time
import gc
import contextlib

import tensorflow
import pandas as pd
import seaborn as sns
import numpy as np

from io import StringIO
from hyperopt import fmin, tpe, hp, Trials, space_eval
from matplotlib import pyplot as plt
from dotenv import load_dotenv
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report

sys.path.append("../")

from scripts.mlflow_functions import train_and_log, train_and_log_keras
from models.scorer import home_credit_scoring_fn, home_credit_scorer, hc_threshold_score
from models.scorer import home_credit_loss_fn_keras


load_dotenv()
sns.color_palette('colorblind')
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

try:
    pc_dpi = int(os.getenv('DPI'))
except TypeError:
    pc_dpi = 100
if pc_dpi is None:
    pc_dpi = 100

client = mlflow.MlflowClient(tracking_uri=os.path.abspath("../mlruns/"))

mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))


In [ ]:
try:
    mlflow.create_experiment(name="home_credit_model")
except mlflow.MlflowException:
    mlflow.set_experiment(experiment_name="home_credit_model")


In [ ]:
df_model = pd.read_pickle(filepath_or_buffer="../data/df_hc_nm.pkl")


In [ ]:
df_model.head()


In [ ]:
df_model.rename(columns={"TARGET": "Loan_granted"}, inplace=True)

target_col = "Loan_granted"


In [ ]:
target_col = "Loan_granted"


# 1 : Modelisation and experimentation

- Threshold optimization : The first xgboost run will be used to adjust the threshold of the scorer
- Models : Ensemble methods, handle missing values and don't require scaling
    - xGBoost
    - CatBoost
    - LightGBM
- Optimization : Best method will be determined via benchmark
    - GridSearchCV
    - Hyperopt
- Logging via MlFlow :
    - Metrics
    - Custom metric tuning
    - AUROC, confusion matrix
    - Global feature importance
    - Model
    - Method of undersampling
    - Version (handling of nans.)

<hr>


In [ ]:
# Splitting the data into training, validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
    df_model.drop(columns=target_col),
    df_model[target_col],
    test_size=0.3,
    random_state=123
    )

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val,
    test_size=0.25,
    random_state=123
    )


## 1.1 xGBoost

### Hyperopt optimization

In [ ]:
xgb_param_space = {
    "n_estimators": hp.choice("n_estimators", np.arange(200, 800, 25, dtype=int)),
    "max_depth": hp.choice("max_depth", np.arange(3, 9, dtype=int)),
    "learning_rate": hp.loguniform("learning_rate", -7, -3),
    "min_child_weight": hp.quniform("min_child_weight", 1, 6, 0.1),
    "reg_alpha": hp.loguniform("reg_alpha", -6, -3),
    "reg_lambda": hp.loguniform("reg_lambda", -6, -3),
    "subsample": hp.uniform("subsample", 0.5, 0.9),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.3, 0.9),
    "nthread": -1
}

def objective(params):
    model = XGBClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        learning_rate=params["learning_rate"],
        min_child_weight=params["min_child_weight"],
        reg_alpha=params["reg_alpha"],
        reg_lambda=params["reg_lambda"],
        subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
        nthread=params["nthread"]
    )
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring=home_credit_scorer)
    loss = -np.mean(scores)
    return {"loss": loss, "status": "ok"}


trials = Trials()

time_start_hopt = time.perf_counter()
best = fmin(objective, space=xgb_param_space, algo=tpe.suggest, max_evals=20, trials=trials)
time_end_hopt = time.perf_counter()

print(f"Hyperopt search time: {time_end_hopt - time_start_hopt} seconds.")


In [ ]:
best_params_xgb = space_eval(xgb_param_space, best)
gc.collect()
print(best_params_xgb)


In [ ]:
mlflow.set_experiment(experiment_name="home_credit_model")

best_params_xgb_log = {
    "colsample_bytree": 0.778655523535136,
    "learning_rate": 0.0054998702766629335,
    "max_depth": 6,
    "min_child_weight": 2.0,
    "n_estimators": 700,
    "reg_alpha": 0.037071627790320846,
    "reg_lambda": 0.006294264780463717,
    "subsample": 0.8644766597785274
    }

xgb_metrics, xgb_clf = train_and_log(
    estimator=XGBClassifier,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    dataset_version="nans_kept_shap_reduced",
    imb_method="near_miss_one",
    na_thresh=0,
    params=best_params_xgb_log,
    model_name="xbgoost_metric_calibration_run",
)


In [ ]:
# There might be overfitting, we will try and adjust it

y_proba = xgb_clf.predict_proba(X_val)

optimal_threshold = hc_threshold_score(y_val, y_proba)

y_pred = (y_proba[:, 1] >= optimal_threshold).astype(int)

score = home_credit_scoring_fn(y_val, y_pred)
print("Score:", score)


In [ ]:
# Double checking the high AUC with classification report

y_val_proba = xgb_clf.predict_proba(X_val)
y_val_pred = xgb_clf.predict(X_val)

report = classification_report(y_val, y_val_pred)
val_score = home_credit_scoring_fn(y_val, y_pred)

print(f"Validation score: {val_score}")
print(report)


### Observations :
With an AUC of .881, it might be considered that the model is overfitting - or the fact that the competition is quite old in a machine learning POV might explain why xGboost is having such high performances.

## 1.2 : Catboost :

In [ ]:
catboost_param_space = {
    "iterations": hp.choice("iterations", np.arange(200, 800, 25, dtype=int)),
    "depth": hp.choice("depth", np.arange(3, 9, dtype=int)),
    "learning_rate": hp.loguniform("learning_rate", -7, -3),
    "l2_leaf_reg": hp.loguniform("l2_leaf_reg", -6, -3),
    "subsample": hp.uniform("subsample", 0.5, 0.9),
    "colsample_bylevel": hp.uniform("colsample_bylevel", 0.3, 0.9),
    "thread_count": -1
    }


def objective_cb(params):
    model = CatBoostClassifier(
        iterations=params["iterations"],
        depth=params["depth"],
        subsample=params["subsample"],
        colsample_bylevel=params["colsample_bylevel"],
        learning_rate=params["learning_rate"],
        l2_leaf_reg=params["l2_leaf_reg"],
        verbose=False,
    )
    model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring=home_credit_scorer)
    loss = -np.mean(scores)
    return {"loss": loss, "status": "ok"}

trials = Trials()

time_start_hopt = time.perf_counter()
best_cb = fmin(objective_cb, space=catboost_param_space, algo=tpe.suggest, max_evals=20, trials=trials)
time_end_hopt = time.perf_counter()

print(f"Hyperopt search time: {time_end_hopt - time_start_hopt} seconds.")


In [ ]:
best_params_cb = space_eval(catboost_param_space, best_cb)
print(best_params_cb)


In [ ]:
mlflow.set_experiment(experiment_name="home_credit_model")

best_params_cb_log = {
    "colsample_bylevel": 0.8492535750204921,
    "depth": 7,
    "iterations": 700,
    "l2_leaf_reg": 0.011953394749318379,
    "learning_rate": 0.018551685647746795,
    "subsample": 0.8297130817489142,
    "verbose": False
    }

cb_metrics, cb_clf = train_and_log(
    estimator=CatBoostClassifier,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    dataset_version="nans_kept_shap_reduced",
    imb_method="near_miss_one",
    na_thresh=0,
    params=best_params_cb_log,
    model_name="catboost_cvalidated",
)


In [ ]:
# Double checking the high AUC with classification report

y_val_proba_cb = cb_clf.predict_proba(X_val)
y_val_pred_cb = cb_clf.predict(X_val)

report_cb = classification_report(y_val, y_val_pred_cb)
val_score_cb = home_credit_scoring_fn(y_val, y_pred)

print(f"Validation score: {val_score_cb}")
print(report_cb)


##### The problem of our generic function doesnt account for catboost specific scoring args, will be addressed at the end of the nb by defining again the functions imported for mlflow_function

# GradientBoostClassifier

In [ ]:
df_model_imp = pd.read_pickle(filepath_or_buffer="../data/df_hc_nm_imputed.pkl")


In [ ]:
gb_param_space = {
    "learning_rate": hp.loguniform("learning_rate", -5, 0),
    "n_estimators": hp.quniform("n_estimators", 50, 500, 1),
    "max_depth": hp.quniform("max_depth", 2, 8, 1),
    "min_samples_split": hp.quniform("min_samples_split", 2, 20, 1),
    "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 10, 1),
    "subsample": hp.uniform("subsample", 0.5, 1),
    "max_features": hp.choice("max_features", ["sqrt", "log2", None])
}

def objective_gb(params):
    model = GradientBoostingClassifier(
        learning_rate=params["learning_rate"],
        n_estimators=int(params["n_estimators"]),
        max_depth=int(params["max_depth"]),
        min_samples_split=int(params["min_samples_split"]),
        min_samples_leaf=int(params["min_samples_leaf"]),
        subsample=params["subsample"],
        max_features=params["max_features"],
        random_state=123
    )
    model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring=home_credit_scorer)
    loss = -np.mean(scores)
    return {"loss": loss, "status": "ok"}

trials = Trials()

time_start_hopt = time.perf_counter()
best_gb = fmin(objective_gb, space=gb_param_space, algo=tpe.suggest, max_evals=20, trials=trials)
time_end_hopt = time.perf_counter()

print(f"Hyperopt search time: {time_end_hopt - time_start_hopt} seconds.")


In [ ]:
best_params_gb = space_eval(catboost_param_space, best_gb)
print(best_params_gb)


In [ ]:
mlflow.set_experiment(experiment_name="home_credit_model")

best_params_gb_log = {

    }

cb_metrics, cb_clf = train_and_log(
    estimator=GradientBoostingClassifier,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    dataset_version="nans_removed_shap_reduced",
    imb_method="near_miss_one",
    na_thresh=0,
    params=best_params_gb_log,
    model_name="gradient_boost_clf_cv",
)


# DNN :
- Using the scaled version of the dataset without nans

In [ ]:
df_model_scaled = pd.read_pickle(filepath_or_buffer="../data/df_hc_nm_imputed_scaled.pkl")


## Scaling the numeric columns for Keras

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_model_scaled.drop(columns=["TARGET"]),
    df_model_scaled["TARGET"],
    test_size=0.3,
    random_state=123,
    stratify=df_model_scaled["TARGET"]
    )

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.25,
    random_state=123,
    stratify=y_train
    )


## DNN : 

- input of size feature len
- 512 dense
- dropout 30%
- 256 dense
- dropout 20%
- Sigmoid for binary clf
- Validation set to track overfitting


In [ ]:
recall = tensorflow.keras.metrics.Recall()

# Input layer
inputs = tensorflow.keras.Input(shape=(X_train.shape[1],))

# Hidden layers
dense_1 = tensorflow.keras.layers.Dense(512, activation="relu")(inputs)
dropout_30 = tensorflow.keras.layers.Dropout(0.3)(dense_1)
dense_2 = tensorflow.keras.layers.Dense(256, activation="relu")(dropout_30)
dropout_20 = tensorflow.keras.layers.Dropout(0.2)(dense_2)

# Output layer
outputs = tensorflow.keras.layers.Dense(1, activation="sigmoid")(dropout_20)

model = tensorflow.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer="adam", loss=home_credit_loss_fn_keras, metrics=[recall])


In [ ]:
model_summary = model.summary()

summary_buffer = StringIO()

with contextlib.redirect_stdout(summary_buffer):
    model.summary()

model_summary = summary_buffer.getvalue()


In [ ]:
start = time.perf_counter()

history = model.fit(
    X_train,
    y_train,
    epochs=35,
    batch_size=64,
    validation_data=(X_val, y_val)
    )

end = time.perf_counter()

training_time = end - start


In [ ]:
def get_best_model(history):
    """
    Returns the best Keras model based on validation loss from the training history.
    """
    val_loss = history.history["val_loss"]
    best_epoch = val_loss.index(min(val_loss))
    best_model = tensorflow.keras.models.clone_model(model)
    best_model.build(input_shape=X_train.shape[1:])
    best_model.compile(
        optimizer='adam',
        loss=home_credit_loss_fn_keras,
        metrics=[tensorflow.keras.metrics.Recall()]
        )
    best_model.set_weights(history.model.get_weights())
    return best_model, best_epoch



In [ ]:
best_model, best_epoch = get_best_model(history=history)


In [ ]:
best_score = best_model.evaluate(x=X_val, y=y_val)[0]
print(f"Best model achieved after : {best_epoch} epochs")


In [ ]:
mlflow.set_experiment(experiment_name="home_credit_model")

metrics, model = train_and_log_keras(
    model=best_model,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    training_time=training_time,
    model_summary=model_summary,
    history=history,
    model_name="DNN_TEST",
    dataset_version="Nans_imputed_scaled",
    imb_method="nm1",
    home_credit_score=best_score,
)
